<h1><center> CT Reporting Rates </center></h1>

In [1]:
import pandas as pd
import numpy as np
import calendar

In [2]:
# Declare the From and To Dates
FromDate = pd.to_datetime("2021-05-01")
ToDate = pd.to_datetime(FromDate) + pd.offsets.MonthEnd()+pd.DateOffset(days=5)
print('From Date: ',FromDate,'To Date: ',ToDate)

From Date:  2021-05-01 00:00:00 To Date:  2021-06-05 00:00:00


### Import the data
- This is data on reporting for all the Active EMR Sites and those that reported in the last 12 months

In [3]:
rr = pd.read_csv("ReportingRates_CT_Raw.csv")
rr.head()

,DisplayMFL,DisplayFacilityName,DisplaySubcounty,DisplayCounty,DisplayMechanism,DisplayAgency,UploadStatus,UploadDate,Upload_monthYear,SiteCode,MPI_SiteCode,UploadDate_MPI,Upload_monthYear_MPI
0,11243,Baobab Clinic - Bamburi Cement,Nyali,MOMBASA,AFYA PWANI,USAID,Not Uploaded Care & Treatment or PKV(MPI),NaN,NaN,NaN,NaN,NaN,NaN
1,24618,DICE ICL BOMET,Bomet Central,BOMET,HJF-SOUTH RIFT VALLEY,DOD,Not Uploaded Care & Treatment or PKV(MPI),NaN,NaN,NaN,NaN,NaN,NaN
2,15891,GK Prisons Dispensary (Busia),Matayos,BUSIA,LVCT PRISONS,CDC,Not Uploaded Care & Treatment or PKV(MPI),NaN,NaN,NaN,NaN,NaN,NaN
3,12046,GK Prison Dispensary (Isiolo),Isiolo,ISIOLO,LVCT PRISONS,CDC,Not Uploaded Care & Treatment or PKV(MPI),NaN,NaN,NaN,NaN,NaN,NaN
4,14090,Simbiri Nanbell Health Centre,Rachuonyo South,HOMA BAY,EGPAF TIMIZA,CDC,Not Uploaded Care & Treatment or PKV(MPI),NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
rr.shape

(12915, 13)

In [5]:
rr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12915 entries, 0 to 12914
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DisplayMFL            12915 non-null  int64  
 1   DisplayFacilityName   12915 non-null  object 
 2   DisplaySubcounty      12915 non-null  object 
 3   DisplayCounty         12915 non-null  object 
 4   DisplayMechanism      12915 non-null  object 
 5   DisplayAgency         12905 non-null  object 
 6   UploadStatus          12915 non-null  object 
 7   UploadDate            12865 non-null  object 
 8   Upload_monthYear      12865 non-null  object 
 9   SiteCode              12865 non-null  float64
 10  MPI_SiteCode          6093 non-null   float64
 11  UploadDate_MPI        6093 non-null   object 
 12  Upload_monthYear_MPI  6093 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.3+ MB


### Data Cleaning
- Convert Dates to correct data types
- Replace NULL with NA's 
- Remove Duplicates (Grouped by MFLCode and UploadDate)

In [6]:
rr=rr.replace({'NULL':np.NaN})

In [7]:
# Columns with NULLS
rr.columns[rr.isna().any()].tolist()

['DisplayAgency',
 'UploadDate',
 'Upload_monthYear',
 'SiteCode',
 'MPI_SiteCode',
 'UploadDate_MPI',
 'Upload_monthYear_MPI']

In [8]:
#convert the upload_date and Upload_date MPI field from object to datetime datatype
rr['UploadDate'] = pd.to_datetime(rr['UploadDate'])
rr['UploadDate_MPI'] = pd.to_datetime(rr['UploadDate_MPI'])

In [9]:
rr.tail()

,DisplayMFL,DisplayFacilityName,DisplaySubcounty,DisplayCounty,DisplayMechanism,DisplayAgency,UploadStatus,UploadDate,Upload_monthYear,SiteCode,MPI_SiteCode,UploadDate_MPI,Upload_monthYear_MPI
12910,25260,Langi Kodera DISPENSARY,Ndhiwa,HOMA BAY,EGPAF TIMIZA,CDC,Both Care & Treatment + PKV(MPI) uploaded,2021-06-02,Jun-21,25260.0,25260.0,2021-06-02,Jun-21
12911,25260,Langi Kodera DISPENSARY,Ndhiwa,HOMA BAY,EGPAF TIMIZA,CDC,Only Care & Treatment Uploaded,2021-05-05,May-21,25260.0,NaN,NaT,NaN
12912,25260,Langi Kodera DISPENSARY,Ndhiwa,HOMA BAY,EGPAF TIMIZA,CDC,Both Care & Treatment + PKV(MPI) uploaded,2021-03-02,Mar-21,25260.0,25260.0,2021-03-02,Mar-21
12913,25260,Langi Kodera DISPENSARY,Ndhiwa,HOMA BAY,EGPAF TIMIZA,CDC,Both Care & Treatment + PKV(MPI) uploaded,2021-02-10,Feb-21,25260.0,25260.0,2021-02-10,Feb-21
12914,26816,Kyumbi CHS DICE,Athiriver,MACHAKOS,CHS NAISHI,CDC,Only Care & Treatment Uploaded,2021-06-07,Jun-21,26816.0,NaN,NaT,NaN


In [10]:
#Remove Duplicates from original dataset - Unique Site with 1 Upload Date, upload MPI
rr = rr.drop_duplicates(subset=["DisplayMFL","UploadDate"])
rr.shape

(12903, 13)

In [11]:
# Create a column called UploadedCT & UploadedMPI
rr['uploadedCT'] = np.where(rr.UploadDate.isnull(),0,1)
rr['UploadedMPI'] = np.where(rr.UploadDate_MPI.isnull(),0,1)

### Summary Statistics
- How many Unique Facilities do we have (Number of Sites - Duplicates)

In [12]:
# Number of Unique EMR Sites - Denominator/ Expected Uploads
len(pd.unique(rr['DisplayMFL']))

1392

<h2><center> Expected Uploads </center></h2>

In [13]:
#Create a data Frame to hold distinct list of sites for the denominator / Baseline EMR Sites
#We will use later for left joining
rr_denom = rr.drop_duplicates(subset=['DisplayMFL'])
rr_denom=rr_denom[['DisplayMFL','DisplayFacilityName','DisplaySubcounty','DisplayCounty','DisplayMechanism','DisplayAgency']]
rr_denom.head()

,DisplayMFL,DisplayFacilityName,DisplaySubcounty,DisplayCounty,DisplayMechanism,DisplayAgency
0,11243,Baobab Clinic - Bamburi Cement,Nyali,MOMBASA,AFYA PWANI,USAID
1,24618,DICE ICL BOMET,Bomet Central,BOMET,HJF-SOUTH RIFT VALLEY,DOD
2,15891,GK Prisons Dispensary (Busia),Matayos,BUSIA,LVCT PRISONS,CDC
3,12046,GK Prison Dispensary (Isiolo),Isiolo,ISIOLO,LVCT PRISONS,CDC
4,14090,Simbiri Nanbell Health Centre,Rachuonyo South,HOMA BAY,EGPAF TIMIZA,CDC


In [14]:
rr_denom.shape

(1392, 6)

<h2><center> CT RECENCY </center></h2>

In [15]:
#Filter the Original RR Data to the specific RR Period (FromDate ToDate)
mask = (rr['UploadDate'] >= FromDate) & (rr['UploadDate'] <= ToDate)
recency=rr.loc[mask]
recency.shape

(2018, 15)

In [16]:
#Number of Duplicates in CT_Recency Those that reported in the reporting month and between 1st and 5th of next month
len(recency['DisplayMFL'])- len(pd.unique(recency['DisplayMFL']))

787

In [17]:
#Remove duplicates.Making Sure we are counting a site only once within the reporting period being looked at
recency = recency.drop_duplicates(subset=["DisplayMFL"])
recency.shape

(1231, 15)

In [18]:
#Join the CT Recency Dataset to the EMR Sites Denominator - #Left Join
result = pd.merge(rr_denom,
                 recency[['DisplayMFL', 'UploadDate','uploadedCT']],
                 on='DisplayMFL', 
                 how='left')
result.shape

(1392, 8)

<h2><center> CT CONSISTENCY </center></h2>

In [19]:
# Define Period to compute 
consist_FromDate=pd.to_datetime(FromDate)-pd.DateOffset(months=2)
print('Consistency Period - From Date: ',consist_FromDate,'To Date: ',ToDate)

Consistency Period - From Date:  2021-03-01 00:00:00 To Date:  2021-06-05 00:00:00


In [20]:
#Filter the data to return - only sites that have Uploaded every month last 3 months
const = (rr['UploadDate'] >= consist_FromDate) & (rr['UploadDate'] <= ToDate)
consistency=rr.loc[const]
consistency.shape

(4200, 15)

In [21]:
#Pivot the Data then add instances of uploads by month, filter only those that uploaded all 3 months
cs=pd.pivot_table(consistency, values='uploadedCT', 
                    columns='Upload_monthYear',
                    index='DisplayMFL')

cs = cs.replace(np.nan, 0)
cs['UploadedConsistently']=cs.sum(axis=1) 
cs = cs.loc[(cs['UploadedConsistently']>=3)] 
cs.reset_index(inplace=True)# Flatten the table

#If Uploaded consistently is NULL then 0 other wise 1
cs['UploadedConsistently'] = np.where(cs.UploadedConsistently.isnull(),0,1)

cs

Upload_monthYear,DisplayMFL,Apr-21,Jun-21,Mar-21,May-21,UploadedConsistently
0,10019,1.0,0.0,1.0,1.0,1
1,10049,1.0,1.0,1.0,1.0,1
2,10055,1.0,1.0,1.0,1.0,1
3,10100,1.0,0.0,1.0,1.0,1
4,10110,1.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...
1047,23414,1.0,1.0,0.0,1.0,1
1048,24200,1.0,1.0,1.0,1.0,1
1049,24339,1.0,1.0,1.0,1.0,1
1050,24969,1.0,0.0,1.0,1.0,1


In [22]:
#Join the Consistency Dataset to the CT Recency - #Left Join
result = pd.merge(result,
                 cs[['DisplayMFL', 'UploadedConsistently']],
                 on='DisplayMFL', 
                 how='left')
result.shape

(1392, 9)

In [23]:
result

,DisplayMFL,DisplayFacilityName,DisplaySubcounty,DisplayCounty,DisplayMechanism,DisplayAgency,UploadDate,uploadedCT,UploadedConsistently
0,11243,Baobab Clinic - Bamburi Cement,Nyali,MOMBASA,AFYA PWANI,USAID,NaT,NaN,NaN
1,24618,DICE ICL BOMET,Bomet Central,BOMET,HJF-SOUTH RIFT VALLEY,DOD,NaT,NaN,NaN
2,15891,GK Prisons Dispensary (Busia),Matayos,BUSIA,LVCT PRISONS,CDC,NaT,NaN,NaN
3,12046,GK Prison Dispensary (Isiolo),Isiolo,ISIOLO,LVCT PRISONS,CDC,NaT,NaN,NaN
4,14090,Simbiri Nanbell Health Centre,Rachuonyo South,HOMA BAY,EGPAF TIMIZA,CDC,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1387,24339,Murang'a Dice,muranga south,MURANG'A,CHS TEGEMEZA PLUS,CDC,2021-05-05,1.0,1.0
1388,24563,Nairobi Adventist Hospital,Westlands,NAIROBI,CHAK CHAP UZIMA,NaN,2021-06-01,1.0,NaN
1389,24969,Lvct Ruaka Dice,Kiambaa,KIAMBU,LVCT DARAJA,CDC,2021-05-31,1.0,1.0
1390,25260,Langi Kodera DISPENSARY,Ndhiwa,HOMA BAY,EGPAF TIMIZA,CDC,2021-06-02,1.0,1.0


### Reporting Rates by County

In [24]:
byCounty=(result.groupby(['DisplayCounty'])
   .agg({'DisplayCounty':'count','uploadedCT': 'sum','UploadedConsistently':'sum'})
   .rename(columns={'DisplayCounty':'ExpectedUpload','uploadedCT' : 'CT Recently Uploaded','UploadedConsistently' : 'CT Consistency' })
         )
byCounty.reset_index(inplace=True)# Flatten the table
byCounty['Recency%']=(byCounty['CT Recently Uploaded']/byCounty['ExpectedUpload'])
byCounty['Consistency%']=(byCounty['CT Consistency']/byCounty['ExpectedUpload'])
byCounty=byCounty [['DisplayCounty','ExpectedUpload','CT Recently Uploaded','Recency%','CT Consistency','Consistency%']]
byCounty

,DisplayCounty,ExpectedUpload,CT Recently Uploaded,Recency%,CT Consistency,Consistency%
0,BARINGO,11,11.0,1.000000,11.0,1.000000
1,BOMET,23,19.0,0.826087,15.0,0.652174
2,BUNGOMA,24,24.0,1.000000,23.0,0.958333
3,BUSIA,23,19.0,0.826087,17.0,0.739130
4,ELGEYO MARAKWET,9,3.0,0.333333,3.0,0.333333
5,EMBU,21,13.0,0.619048,10.0,0.476190
6,HOMA BAY,170,166.0,0.976471,100.0,0.588235
7,ISIOLO,1,0.0,0.000000,0.0,0.000000
8,KAJIADO,20,18.0,0.900000,18.0,0.900000
9,KAKAMEGA,52,47.0,0.903846,47.0,0.903846


### Reporting Rates by Partner

In [25]:
byPartner=(result.groupby(['DisplayMechanism'])
   .agg({'DisplayMechanism':'count','uploadedCT': 'sum','UploadedConsistently':'sum'})
   .rename(columns={'DisplayMechanism':'ExpectedUpload','uploadedCT' : 'CT Recently Uploaded','UploadedConsistently' : 'CT Consistency' })
         )
byPartner.reset_index(inplace=True)# Flatten the table
byPartner['Recency%']=(byPartner['CT Recently Uploaded']/byPartner['ExpectedUpload'])
byPartner['Consistency%']=(byPartner['CT Consistency']/byPartner['ExpectedUpload'])
byPartner=byPartner [['DisplayMechanism','ExpectedUpload','CT Recently Uploaded','Recency%','CT Consistency','Consistency%']]

### Save the data to Excel

In [26]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
rrperiod=str(calendar.month_abbr[FromDate.month])+str(FromDate.year)
filename='CTReportingRates_'+str(rrperiod)+".xlsx"

In [30]:
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
result.to_excel(writer, sheet_name='LineList')
byCounty.to_excel(writer, sheet_name='ByCounty')
byPartner.to_excel(writer, sheet_name='ByPartner')

# Close the Pandas Excel writer and output the Excel file.
writer.save()